<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_192Vectors_Group_Data_SectorEmbed_WeightedLoss_Debayan_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! rm -rf /content/idl-spring-22-project-deepar
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1852, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1852 (delta 9), reused 0 (delta 0), pack-reused 1829
Receiving objects: 100% (1852/1852), 82.50 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (886/886), done.
Checking out files: 100% (1947/1947), done.


In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
! mkdir -p '/content/drive/MyDrive/DeepARExperiments/'

In [4]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.8 MB 14.0 MB/s 
     |████████████████████████████████| 181 kB 63.7 MB/s 
     |████████████████████████████████| 144 kB 67.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=5523d5e371b362d0ddbabb6f5e3e9bbe6053c4bbdec86479bc8816999392ef0d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
import wandb

wandb.init(project="vector-training", entity="idl-deepar-spring-22", name="baseline-more-data-weighte-loss")

wandb: Currently logged in as: mmml (use `wandb login --relogin` to force relogin)


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [7]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_random_groups.py -gs=5 -r=50000 -w=192
! pip install kaleido

total rows:  0
total rows:  95
total rows:  613
total rows:  1262
total rows:  3851
total rows:  3851
total rows:  3851
total rows:  5667
total rows:  5667
total rows:  7786
total rows:  7786
total rows:  7786
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in subtract
  return (a - mns) / sstd
found nans
total rows:  7786
total rows:  7786
total rows:  7844
total rows:  7938
total rows:  8826
total rows:  9179
total rows:  9896
total rows:  9896
total rows:  10382
total rows:  10392
total rows:  10392
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in subtract
  retu

In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/visualize.py

tcmalloc: large alloc 1233903616 bytes == 0x55ddcb8ac000 @  0x7f5be213f615 0x55dd53e89057 0x55dd53f347b6 0x55dd53f32e59 0x55dd53e1e840 0x55dd53f91868 0x55dd53e7e49a 0x55dd53ef1464 0x55dd53eeaa2e 0x55dd53e7d88a 0x55dd53eec719 0x55dd53eeaa2e 0x55dd53e7d88a 0x55dd53eec719 0x55dd53eeaa2e 0x55dd53eea723 0x55dd53fb4812 0x55dd53fb4b8d 0x55dd53fb4a36 0x55dd53f8c183 0x55dd53f8be2c 0x7f5be0f28c87 0x55dd53f8bd0a
^C


In [8]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/sector_company_map.pkl','rb') as f:
  sector_company_map = pickle.load(f)

In [9]:
sector_to_index = {}
index_to_sector = {}

for i, sector_name in enumerate(sorted(sector_company_map.keys())):
    sector_to_index[sector_name]  = i
    index_to_sector[i] = sector_name

print(sector_to_index)
print(index_to_sector)

{'Basic Materials': 0, 'Communication Services': 1, 'Consumer Cyclical': 2, 'Consumer Defensive': 3, 'Energy': 4, 'Financial Services': 5, 'Healthcare': 6, 'Industrials': 7, 'Real Estate': 8, 'Services': 9, 'Technology': 10, 'Utilities': 11}
{0: 'Basic Materials', 1: 'Communication Services', 2: 'Consumer Cyclical', 3: 'Consumer Defensive', 4: 'Energy', 5: 'Financial Services', 6: 'Healthcare', 7: 'Industrials', 8: 'Real Estate', 9: 'Services', 10: 'Technology', 11: 'Utilities'}


In [10]:
print(index_to_company)
print(company_to_index)
print(sector_company_map)
print(sector_company_map.keys())
print(sector_company_map['Technology'])
print(len(sector_company_map))

{0: 'GEVO', 1: 'MP', 2: 'WWR', 3: 'MAG', 4: 'EXK', 5: 'LAC', 6: 'AG', 7: 'CC', 8: 'AU', 9: 'CVNA', 10: 'OSTK', 11: 'HLT', 12: 'JD', 13: 'CPRI', 14: 'CLF', 15: 'VALE', 16: 'NG', 17: 'SHOP', 18: 'TTCM', 19: 'AMD', 20: 'CFLT', 21: 'BB', 22: 'VRAR', 23: 'WEJO', 24: 'TXN', 25: 'DM', 26: 'NEWR', 27: 'VET', 28: 'VLO', 29: 'DNN', 30: 'SLB', 31: 'GTE', 32: 'AGR', 33: 'OEG', 34: 'D', 35: 'HLGN', 36: 'PCG', 37: 'UNP', 38: 'LQMT', 39: 'R', 40: 'RCRT', 41: 'LMT', 42: 'ES', 43: 'PPGH', 44: 'SOFI', 45: 'C', 46: 'LMND', 47: 'BAC', 48: 'IVZ', 49: 'RELI', 50: 'BX', 51: 'DEFTF', 52: 'CNGT', 53: 'EBAY', 54: 'LCID', 55: 'F', 56: 'PTRA', 57: 'ORGN', 58: 'FCX', 59: 'MOS', 60: 'HCC', 61: 'FREE', 62: 'COST', 63: 'KO', 64: 'WMT', 65: 'FAMI', 66: 'AFMD', 67: 'CLDX', 68: 'GERN', 69: 'SYK', 70: 'SXTC', 71: 'POLA', 72: 'UAL', 73: 'J', 74: 'BA', 75: 'GOOGL', 76: 'ATUS', 77: 'SBGI', 78: 'GRPN', 79: 'SPOT', 80: 'OPAD', 81: 'AHT', 82: 'MSTO', 83: 'OPEN', 84: 'RDFN', 85: 'IRTC', 86: 'BLUE', 87: 'BIIB', 88: 'RVPH', 89: '

In [11]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [12]:
NUM_TRAIN_SAMPLES = len(stock_inputs)
NUM_TEST_SAMPLES  = len(stock_test_inputs)

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [13]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (50075, 192, 13)
stock_labels_trimmed shape (50075, 192, 13)
stock_test_inputs_trimmed shape (13138, 192, 13)
stock_test_labels_trimmed shape (13138, 192, 13)


In [14]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

def plot_data(data, window_id,cov_id):
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  df = pd.DataFrame()
  df['data'] = data[window_id,:,cov_id]
  

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['data'], mode = 'lines', name = 'data'))

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Y')
  fig.show()

In [15]:
plot_data(stock_inputs_trimmed, 20000, 12)

In [26]:


def generate_horizon_weights(horizon_len):

  weight = [1]
  for i in range(1, horizon_len):
    weight.append(weight[-1]*2)

  weight_arr = np.array(weight)

  return weight_arr/ np.sum(weight_arr)

HORIZON_PERIOD = 5

params = {
    'num_classes': len(sector_company_map),
    'embedding_dim':len(sector_company_map),
    #  month, day, 5 companies open and volume
    'cov_dim': 7,
    'op_dim': 5,  # op does not have the "time covariates"
    'lstm_hidden_dim': 128,
    'lstm_layers':4,
    'window_size':192,
    'batch_size': 64,
    'learning_rate': 0.005,
    'epochs':100,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'num_companies': 5,
    'mu_sigma_dim': 1,  # each datapoint is one-d,
    'scheduler_step_size': 20,
    'scheduler_gamma': 0.95,
    'horizon_period': HORIZON_PERIOD,
    'horizon_weights': generate_horizon_weights(HORIZON_PERIOD)
}
# wandb.config = params
print(params)

{'num_classes': 12, 'embedding_dim': 12, 'cov_dim': 7, 'op_dim': 5, 'lstm_hidden_dim': 128, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 64, 'learning_rate': 0.005, 'epochs': 100, 'num_test_samples': 13138, 'num_train_samples': 50075, 'conditioning_period': 168, 'prediction_period': 24, 'num_companies': 5, 'mu_sigma_dim': 1, 'scheduler_step_size': 20, 'scheduler_gamma': 0.95, 'horizon_period': 5, 'horizon_weights': array([0.03225806, 0.06451613, 0.12903226, 0.25806452, 0.51612903])}


In [27]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs
        self.label = stock_labels
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
        x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))

        win_len, ip_dim = x1.shape

        sector_name = self.data[index][0,12]
        sector_id = sector_to_index[sector_name]

        sector_id_concat = sector_id * torch.ones((win_len, 1))
        x1 = torch.cat((x1, sector_id_concat), dim=1)
        return x1, x2  

class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs
        self.label = stock_test_labels
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
      x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))

      # Add Sector ID to the input
      win_len, ip_dim = x1.shape

      sector_name = self.data[index][0,12]  # 12 is company index id
      sector_id = sector_to_index[sector_name]

      # sector_id_concat -> Sector Id tensor that is to be concatenated to x1
      sector_id_concat = sector_id * torch.ones((win_len, 1))
      x1 = torch.cat((x1, sector_id_concat), dim=1)

      return x1, x2  

In [28]:
train_dataset = TrainDataset()

for x1, x2 in train_dataset:

    print("data shape ")
    print(x1[:,-1])
    break


data shape 
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [29]:
class TrainDatasetHorizon(Dataset):
    def __init__(self):
        self.data = stock_inputs
        self.label = stock_labels
        self.train_len = self.data.shape[0]
        self.horizon_period = params['horizon_period']


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
        x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))
        

        # Add Sector ID to the input
        win_len, ip_dim = x1.shape

        sector_name = self.data[index][0,12]  # 12 is company index id
        sector_id = sector_to_index[sector_name]

        # sector_id_concat -> Sector Id tensor that is to be concatenated to x1
        sector_id_concat = sector_id * torch.ones((win_len, 1))
        x1 = torch.cat((x1, sector_id_concat), dim=1)

        # Add Horizon Information
        x_shift = [None]*self.horizon_period
        x_shift = [torch.roll(x2, shifts=-1*i, dims=0)[:-1*(self.horizon_period - 1 )] for i in range(0, self.horizon_period)]

        return x1[:-1*(self.horizon_period - 1 ), :], x_shift  

In [30]:
train_horizon = TrainDatasetHorizon()

In [31]:
for x1, x2 in train_horizon:

    print("---x1---", x1[:, 7])

    break

---x1--- tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [32]:
train_data = TrainDatasetHorizon()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)


In [33]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sqrt((1/(N)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result


# MODEL


In [34]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        
        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        self.price_mu = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'], params['num_companies']),
                        nn.GELU()
                      )
        
        self.price_sigma = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'], params['num_companies']),
                        nn.Softplus()
                      )
        
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])
        

    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''

        cov = x[:, :, :-1]   # remove the sector index from the inputs to get the covariates
        sector_index = x[:, 0, -1].to(torch.int32)  # retrieve the sector index from the covariates
        onehot_embed = self.embedding(sector_index)

        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_price_mu = self.price_mu(out1)
        out_price_sigma = self.price_sigma(out1)
        
        return out_price_mu, out_price_sigma, hn_cn



In [35]:
model = Network(params)
print(model)

Network(
  (lstm): LSTM(19, 128, num_layers=4, batch_first=True)
  (price_mu): Sequential(
    (0): Linear(in_features=128, out_features=5, bias=True)
    (1): GELU()
  )
  (price_sigma): Sequential(
    (0): Linear(in_features=128, out_features=5, bias=True)
    (1): Softplus(beta=1, threshold=20)
  )
  (embedding): Embedding(12, 12)
)


In [36]:
for ip, op in train_loader:
    print("ip shape", ip.shape)
    print("op shape", op[0].shape)

    fwd = model.forward(ip)
    print("fwd shape", fwd[0].shape)
    break

ip shape torch.Size([64, 188, 8])
op shape torch.Size([64, 188, 5])
fwd shape torch.Size([64, 188, 5])


In [37]:
def gauss_fit_and_sample(price_mu, price_sigma):
  window_len = price_mu.shape[1]
  batch_size = price_mu.shape[0]

  # BATCH, SEQ_LEN, NUM_COMPANIES, 1
  price_mu_reshape = price_mu.view(batch_size * window_len * params['num_companies'], params['mu_sigma_dim'])
  price_sigma_reshape = price_sigma.view(batch_size * window_len * params['num_companies'], params['mu_sigma_dim'])

  gauss_price = torch.distributions.Normal(price_mu_reshape, price_sigma_reshape)

  # BATCH, SEQ_LEN, NUM_COMPANIES, 1
  price_sampled_reshape = gauss_price.sample()
  price_sampled = price_sampled_reshape.view(batch_size, window_len, params['num_companies'])

  return price_sampled

In [38]:
mu_rand = torch.rand(2, 3, 5)
sigma_rand = torch.ones((2,3,5))* 0.000000001

sampled_data = gauss_fit_and_sample(mu_rand, sigma_rand)

print(mu_rand)
print(sampled_data)


tensor([[[0.0335, 0.5192, 0.5797, 0.1589, 0.1398],
         [0.8942, 0.3128, 0.1560, 0.1026, 0.5984],
         [0.2966, 0.9490, 0.1602, 0.8294, 0.6615]],

        [[0.1798, 0.5280, 0.7804, 0.4995, 0.2518],
         [0.3930, 0.7531, 0.6469, 0.8505, 0.7630],
         [0.1457, 0.3878, 0.8107, 0.0439, 0.0806]]])
tensor([[[0.0335, 0.5192, 0.5797, 0.1589, 0.1398],
         [0.8942, 0.3128, 0.1560, 0.1026, 0.5984],
         [0.2966, 0.9490, 0.1602, 0.8294, 0.6615]],

        [[0.1798, 0.5280, 0.7804, 0.4995, 0.2518],
         [0.3930, 0.7531, 0.6469, 0.8505, 0.7630],
         [0.1457, 0.3878, 0.8107, 0.0439, 0.0806]]])


In [39]:
.0# Test Code Conditioning Period + Prediction Period


def validate(model):

  model.eval()


  conditionining_period = params['conditioning_period']
  prediction_period     = params['prediction_period']

  assert conditionining_period + prediction_period == params['window_size']

  with torch.no_grad():
      
    overall_price_mu = None
    # overall_volume_mu = None

    overall_price_sigma = None
    # overall_volume_sigma = None

    overall_price_label = None
    # overall_volume_label = None

    overall_price_pred = None
    # overall_volume_pred = None

    for i, (ip_covariate, op_label) in enumerate(test_loader):

      ip_covariate = ip_covariate.cuda()
      op_label = op_label.cuda()

      cond_ip = ip_covariate[:, 0:params['conditioning_period'], :]
      cond_op = op_label[:, 0:params['conditioning_period'], :]

      pred_ip = ip_covariate[:, params['conditioning_period']:, :]
      pred_op = op_label[:, params['conditioning_period']:, :]

      # Step One - Forward Pass : Conditioning Period.
      price_mu, price_sigma, ht_ct = model.forward(x=cond_ip, h0_c0=None)

      # Sample GMM and get PRICE/ volume
      price_sampled = gauss_fit_and_sample(price_mu, price_sigma)

      price_sampled_last = price_sampled[:, -1, :].unsqueeze(1)

      batch_price_mu = price_mu
      batch_price_sigma = price_sigma

      batch_price_pred = price_sampled

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)

        # pred_cov_ip[:, 0, 2:2 + params['num_companies']] = price_sampled_last[:, 0, :]
        pred_cov_ip[:, 0, 0:params['num_companies']] = price_sampled_last[:, 0, :]
        price_pred_mu, price_pred_sigma, ht_ct = model.forward(x=pred_cov_ip,h0_c0=ht_ct)
        price_sampled_last = gauss_fit_and_sample(price_pred_mu, price_pred_sigma)

        batch_price_mu = torch.cat((batch_price_mu, price_pred_mu), dim=1)
        batch_price_sigma = torch.cat((batch_price_sigma, price_pred_sigma), dim=1)        
        batch_price_pred = torch.cat((batch_price_pred, price_sampled_last), dim=1)

      overall_price_mu = batch_price_mu if overall_price_mu is None else torch.cat((overall_price_mu, batch_price_mu), dim=0)
      overall_price_sigma = batch_price_sigma if overall_price_sigma is None else torch.cat((overall_price_sigma, batch_price_sigma), dim=0)
      overall_price_pred = batch_price_pred if overall_price_pred is None else torch.cat((overall_price_pred, batch_price_pred), dim=0)
      overall_price_label = op_label[:, :, 0:params['num_companies']] if overall_price_label is None else torch.cat((overall_price_label, op_label[:, :, 0:params['num_companies']]), dim=0)
    
  
  return overall_price_pred,overall_price_label, overall_price_mu, overall_price_sigma


In [40]:
def loss_fn(out_price_mu, out_price_sigma, op_label):

    B, S, D = out_price_mu.shape

    out_price_mu = out_price_mu.view(B * S * params['num_companies'], params['mu_sigma_dim'])
    out_price_sigma = out_price_sigma.view(B * S * params['num_companies'], params['mu_sigma_dim'])
    
    gauss_price =  torch.distributions.Normal(out_price_mu, out_price_sigma)
    
    op_label_price = op_label.view(B * S * params['num_companies'], params['mu_sigma_dim'])

    sample_loss =  -1 * torch.sum(gauss_price.log_prob(op_label_price)) / (B * S * params['num_companies'])

    return sample_loss


In [41]:
def loss_fn_v2(out_price_mu, out_price_sigma, op_label):

    B, S, D = out_price_mu.shape

    loss = 0
    for b in range(0, B):
      for seq in range(0, S):
        mean = out_price_mu[b, seq].view(-1, 1)
        sigma = out_price_sigma[b, seq].view(-1, 1)
        op    = op_label[b, seq].view(-1, 1)

        gauss =  torch.distributions.Normal(mean, sigma)
        # print("mean", mean.shape, "sigma", sigma.shape, "op", op.shape)
        loss += -1 * torch.sum(gauss.log_prob(op))
    return  loss / (B * S * params['num_companies'])



In [42]:
mu_rand = torch.rand(2, 3, 5)
sigma_rand = torch.rand(2, 3, 5)
op_rand = torch.randn(2, 3, 5)

loss_val = loss_fn(mu_rand, sigma_rand, op_rand)
loss_val_v2 = loss_fn_v2(mu_rand, sigma_rand, op_rand)

print("loss fn ", loss_val, "loss val v2 ", loss_val_v2)

sigma_rand = 0.000000001 * torch.ones((2, 3, 4))


# print("mu shape", mu_rand.shape)

loss fn  tensor(18.5288) loss val v2  tensor(18.5288)


In [44]:
model = Network(params)

model_version='vectorized_gaussian_weighted_loss_sector_embed_v3.pt'
epochs = 100

optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=params['scheduler_step_size'], gamma=params['scheduler_gamma'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)

for epoch in range(0, epochs):

  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
  model.train()
  model.cuda()

  total_loss = 0
  total_price_loss = 0
  total_volume_loss = 0

  if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
      # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

      checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      epoch = checkpoint['epoch'] + 1
  
  for i, (ip_covariate, op_label_list) in enumerate(train_loader):

    optimizer.zero_grad()

    ip_covariate = ip_covariate.cuda()
    op_label_list     = [op_label.cuda() for op_label in op_label_list]

    
    out_price_mu, out_price_sigma, ht_ct = model.forward(x=ip_covariate)
    
    batch_size = out_price_mu.shape[0]
    seq_with_horizon = params['window_size'] - params['horizon_period'] + 1

    out_price_mu = out_price_mu.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])
    out_price_sigma = out_price_sigma.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])

    loss = 0

    for i, op_label in enumerate(op_label_list):

      op_label_price = op_label.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])
      gauss_price =  torch.distributions.Normal(out_price_mu, out_price_sigma)   

      sample_loss =  -1 * torch.sum(gauss_price.log_prob(op_label_price)) / (batch_size * seq_with_horizon * params['num_companies'])
      loss += sample_loss * params['horizon_weights'][i]
    loss.backward()

    total_loss += float(loss)
    optimizer.step()

    batch_bar.set_postfix(
    loss="{:.04f}".format(float(total_loss / (i + 1))),
    lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

    batch_bar.update() 

  batch_bar.close() # You need this to close the tqdm bar
  overall_price_pred, overall_price_label, overall_price_mu, overall_price_sigma = validate(model)
  scheduler.step(loss)

  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': total_loss/len(train_loader),
          },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')


  rmse_price = rmse(overall_price_pred.view(-1, 1), overall_price_label.view(-1, 1))
  nd_price = ND(overall_price_pred.view(-1, 1), overall_price_label.view(-1, 1))
  wandb.log({"loss":loss})
  print(f"Epoch {epoch}, price loss {loss}")

Epoch 0, price loss 0.11511141806840897


Epoch 1, price loss -0.1908157467842102


Epoch 2, price loss -0.31682145595550537


Epoch 3, price loss -0.44593966007232666


Epoch 4, price loss -0.512670636177063


...

Epoch 28, price loss -0.982041597366333


Epoch 29, price loss -0.43430960178375244


Epoch 30, price loss -0.7194937467575073


Train:  40%|████      | 315/783 [00:16<00:24, 19.01it/s, loss=-55.3008, lr=0.0010]

KeyboardInterrupt: ignored

In [45]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

def plot_test_output_data(actual,predicted, window_id,cov_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id,:,cov_id]
  df['predicted'] = predicted[window_id,:,cov_id]
  

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()
  


In [52]:
for i in range(5):
  plot_test_output_data(overall_price_label.detach().cpu(),overall_price_pred.detach().cpu(), window_id=3000, cov_id=i)